# Example use of OptiMG DAT RAMP model

## Input data
- Input is provided as dict and must be generate in the previous steps of the DAT from survey response data
- Each user (e.g. household, business, farm...) has a entry in the input data dict.
- From each survey response one user entry should be generated
- With each user entry contains a dict with information specific to one of the five demands (electrical appliances, agro-processing, cooking, drinking water, service water). If the user does not have the corresponding demand, an empty dict must be passed.
- A dict containing admin data must be provided.

In [ ]:
from input.complete_input import input_dict
from input.admin_input import admin_input

## Create instance of RampControl
- RampControl contains the custom RAMP model for all 5 demands.
- A start date (in datetime string format: e.g. '2018-01-01') and the number of days to model must be specified.

In [ ]:
from ramp_model.ramp_control import RampControl
ramp_control = RampControl(365, '2018-01-01')

## Generate demand profiles
- All functions to pass input data to the RAMP model and to run the model are combined in one function for application in OptiMG.
- It returns a multi-level columns dataframe containing all modeled demand profiles (resampled to hourly resolution) with the following shape:
- Demand name 1-5 (cooking, drinking water...) -> User A, B, C... -> Demand (e.g. individual electrical appliance, irrigation water, lunch cooking...)
- Users and individual demand names correspond to passed names in input dict

In [ ]:
dat_output = ramp_control.run_opti_mg_dat(input_dict, admin_input)
dat_output.head()

## Exemplary plotting
- Individual plot for each demand type

In [ ]:
from helpers import plotting
from plotly.subplots import make_subplots

fig = make_subplots(rows=5, cols=1, shared_xaxes=True)

i = 1  # Plotly subplot rows start at index 1
for demand, df in dat_output.groupby(level=0, axis=1):
    fig = plotting.plotly_high_res_df(fig, df=df, subplot_row=i)
    i = i + 1
    print(demand)

fig.update_layout(autosize=True)

fig.show_dash(mode='inline')

## Aggregating demands
- The individual demands can easily be aggregated as needed
- For example by demand type (for water demands this might not be applicable because the individual demands might have different pumping heads. To be considered in preprocessing of system modeling)

In [ ]:
agg_demand = dat_output.groupby(level=0, axis=1).sum()
agg_demand.head()

fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig = plotting.plotly_high_res_df(fig, df=agg_demand, subplot_row=1)
fig.show_dash(mode='inline')